## COLONIAL DATA MINING PROJECT

### Introduction and PDF Reading

I recently enrolled in a political science class which requires me to perform somesort of empirical analysis pertaining to african colonies. After deciding that I wanted to do something pertaining to the growth of export economy in colonial dependencies, I quickly found a dearth of easily-accessible data in convenient formats (e.g. JSON, CSV, etc.). Instead, I found several dozen PDFs originating from the british colonial office, each of which enumerated exported commodities and corresponding monetary values. (For those interested, they can locate these documents by searching on archive.org for documents authored by the "British colonial office" from between 1920 and 1938. I chose this time period specifically, because it was the most thoroughly cataloged.)

Anyone familiar with PDF parsing can predict some of the issues ahead. As an SO user told me during the initial stages of this project, "the first rule of pdf parsing is don't". Unfortunately, the lack of more accessible data formats forced me to pursue PDF parsing. I gradually added to a jupyter notebook file over the course of a couple days until it reached a truly grotesque size (the code approaches around 600 lines total, regrettably making this my largest repo at the moment). It's definitely some of the messier code I've written, but the script functions with a relatively high degree of accuracy. Having recently learned the basics of jupyter's markdown notation, I thought it would be a valuable experience to retroactively annotate my work.

To begin, let's extract text from each pdf (all stored in the 'pdf' directory) using poppler.

In [10]:
import os
import re
import PyPDF2 as pdf
from multiprocessing import Pool
from poppler import load_from_file, PageRenderer

#Take a file from 'pdfs' dir, return arr of its pages' texts.
def getText(file):
    print(file)
    doc = load_from_file('pdfs/' + file)
    pages = []
    pageNum = 0
    
    while (pg := doc.create_page(pageNum)):
        try:
            pages.append(pg.text())
            pageNum += 1
        #invalid page index.
        except AttributeError:
            break

    return pages

pdfs = [file.name for file in os.scandir('pdfs')]
texts = { pdf : getText(pdf) for pdf in pdfs }

b31409350.pdf
b31411514.pdf
b31409106.pdf
b31409404.pdf
b31408990.pdf
b31409362.pdf
b31410327.pdf
b31411472.pdf
b31409052.pdf
b31409064.pdf
b3141283x.pdf
b31412865.pdf
b31409325.pdf
b31409386.pdf
b31409398.pdf
b31409374.pdf
b31411113.pdf
b3140943x.pdf
b31411071.pdf
b31410315.pdf
b31411101.pdf
b31409003.pdf
b31411484.pdf
b31409349.pdf
b31409076.pdf
b31409015.pdf
b31409416.pdf
b31410273.pdf
b31411083.pdf
b31411095.pdf
b31408989.pdf
b31409337.pdf
b31409027.pdf
b31409301.pdf
b31409040.pdf
b31410297.pdf
b31409428.pdf
b31411058.pdf
b31409313.pdf
b3140909x.pdf
b31409088.pdf
b31410261.pdf
b31411502.pdf
b31409118.pdf
b31411496.pdf
b31409039.pdf
b31409295.pdf
b3141106x.pdf


### Extracting Export Tables from PDF text

Each pdf contains a single table of exports. There are several formats, but we'll offer a brief example to show what's ahead of us:

```
                                                       1933.                       19 34.
              Article.
                                              Quantity.        Value.   Quantity.           Value.
                                               Tons              £        Tons                £
CoCOQ/       •••       •••   •••   •••         5,435           88,013    4,877              69,614
Coffee, raw ...                                    3              135        1                  29
Cotton, raw ...                                   36              363       77                 906
Grain—maize                                        2               23          1                 3
Kola-nuts                                         20              488       15                 180
Palm kernels                                      88              517      137                 567
Other articles                                   —              6,575      —                 6,844
                   Total     .                   —         £96,114         —           £78,143
                   
```

This foreshadows several deficiencies of the pdf parser that we'll have to deal with. Notably, it has trouble reading pdfs, inserts arbitrary spaces, has a hard type retaining formatting, and confuses similar characters (i.e. '...' -> '•••'). These are, for the most part, surmountable, so we'll put a pin in that for now.

Right now, the task at hand is to distinguish tables that list exports from the surrounding text and, more importantly,from other tables. The most obvious criterion would be to select tables which are in close proximity of the word 'export'. They should also have a list of years at the top separated by large spaces, though we need to make sure that our years regex is relatively general to account for parser errors.

In [13]:
#DEFINING REGEXES.
chars = '.+'

exportText = '(export.{0,200})'
yearRgx = '((19[0-9b ]{2}).{,2}[\s]+)'

#There will be a second rgx to account for cases that this does not cover.
expTabRgx1 = '(' + exportText + yearRgx + '{2,}(' + chars + '))'

Our above regex captures the entire page after signs that the page will contain an export table (namely, the word 'export', followed by a sequence of at most 200 characters, followed by a list of space-demarcated years in the table's header). From that, we can make sure that we capture all of the table on that page. At this point, we'll need to search through the page's content to disaggregate the table from everything else.

We can develop a rough heuristic for whether a line belongs to a table by observing the presence of large spaces. Of course, many lines that are contained within a table don't match this; perhaps the line is a label for a commodity which has been bifurcated across multiple lines (e.g. ex. 1), or perhaps we're dealing with a quirk of the pdf parser (ex. 2).


#### EX. 1:

```
Skins, sheep and
  goat ...             No.         _            _
                                                       952,494 1,559,272 1,566,115
```

#### EX 2:

```
Tobacco 2,303 257,998 3,145 352,348 3,088 345,872 4,081 457,122 6,905 780,964
```

As a result, we're not just looking for table lines; we're looking for an area containing a high concentration of table lines.  To define a regex for tables, we'l look for at a set of table lines separated by at most 1 broken or non-table lines. (A broken line is a line which uses the term '¬' to break up a word.) These are just heuristics, we'll still have plenty of filtering to do later.

In [14]:
#At least 2 repetitions of ' '.
largeSpaces = ' {2,}'


#Characters separated by large spaces, as is the case in tables.
tableLine = '(\n?(' + chars + largeSpaces + chars + ')\n?)'

#Just a line of characters.
nonTableLine = '(\n?(' + chars + ')\n?)'
#Pdf reader denotes dash to s
brokenLine = '(.+¬\n.)'
brokOrNonTab = brokenLine + '?' + nonTableLine + '{0,2}'

#A table is defined as a set of table lines separated by
#at most 1 non-table lines.
tableRgx = '((' + tableLine + '+' + brokOrNonTab + ')+)'

I also mentioned that our initial export table regex wouldn't match all cases. We'll need a second to account for a handful of exceptions.

We'll make a function to take a page worth of text as input, search for each of the regexes, and return the results matching the parameters we've outlined as indicative of an export table.

In [15]:
expTabRgx2 = '(' + exportText + tableRgx + ')'

#It is necessary to parse pages with multiple regexes, since the first may fail.
def getExportPage(pageText, expTabRgx1, expTabRgx2):
    results1 = [res[0] for res in
                re.findall(expTabRgx1, pgText,
                re.IGNORECASE | re.DOTALL) ]
    results2 = [res[0] for res in
                re.findall(expTabRgx2, pgText,
                re.IGNORECASE | re.DOTALL) ]
    
    results = results1 if results1 else results2
    
    #If a table is sideways, we can't use it. The parser produces gibberish.
    #For some reason, the '®' pops up a lot in sideways tables, so we'll filter it out.
    isSideways = any(['®' in res for res in results])
    
    if isSideways:
        return []
    
    return results

After this, we'll define a couple more functions that will be useful when going through our pdfs page-by-page. `filterRes` in particular will help us on multiple occasions where we need to determine if a string matches certain characteristics.

In [12]:
#Gets last line of string passed to it.
def getLastLine(res):
    try:
        return res.split('\n')[-2]
    except:
        return res.split('\n')[-1]

#Does res include at least one term from each list in 'includeSets'?
#Does it also not contain any terms from any list in 'excludeSets'?
def filterRes(res, includeSets, excludeSets):
    includeVals = []
    
    for termList in includeSets:
        includeVals.append(
            any([re.search(term, res, re.IGNORECASE)
                 for term in termList])
        )
        
    excludeVals = []
    
    for termList in excludeSets:
        excludeVals.append(
            any([re.search(term, res, re.IGNORECASE)
                 for term in termList])
        )
       
    return (all(includeVals) and not any(excludeVals))

We still need to narrow our search a bit to ensure that we generally receive 1 table per pdf. To do this, we'll make sure that the table includes the name of at least 1 export ('bannanas, 'obacco' (the pdf parser occasionally confuses 'tobacco' with 'obacco'), 'copper', 'hide', and 'kola'). We'll also make sure that it's not an unrelated table being confused with a pdf table by excluding certain terms: by excluding 'ales' and 'cigar', we ensure that no import tables slip through the cracks; by excluding 'tariff', we filter out tarriff tables; 'seiz' excludes tables enumerating property confiscated by colonial authorities.

Lastly, before we begin our search, we'll need to separate tables which are separated by a single paragraph. Let's make a regex to identify long lines of text separated by single spaces. This particular regex needed to be tailored to a surprising degree of specificity to meet all relevant cases, so I won't walk through each component.

In [17]:
expList = ['bananas', 'obacco', 'copper', 'hide', 'kola']
termsToExclude = ['tariff', 'ales', 'cigar', 'seiz']

#A line with single spaces that does not contain 'tobacco', because there's
#one line containing 'tobacco' which would break this otherwise.
singSpaceLineRgx = '((?!Tobacco)^ *(?:[^ \n]+ [^ \n]){5,}(?:(?!  ).)* *$)+'

Now that we've developed a robust set of criteria to identify and isolate export tables, let's loop through each page of each pdf and find the export tables.

In [65]:
expTabsRaw = { pdf: [] for pdf in texts }

for pdf, textList in texts.items():
    skipNext = 0
    processedPages = set()
    
    for page, pgText in enumerate(textList):
        #If the last loop already parsed this page.
        if page in processedPages:
            continue
        
        results = getExportPage(pgText, expTabRgx1, 
                                expTabRgx2)

        for res in results:
            #If the last line is a table line, the table will continue to next page.
            #In that case, add the next page's content to this page and parse as single table.
            lastLine = getLastLine(res)

            if re.search(tableRgx, lastLine):
                try:    
                    res = res + textList[page + 1]
                    processedPages.add(page + 1)
                    
                except:
                    pass
            
            cleanRes = re.sub('\n{2,}', '\n', res)

            tableRes = [res[0] for res in 
                        re.findall(tableRgx, cleanRes)]
            
            for tabInd, tab in enumerate(tableRes):
                #Split by any line that seems like paragraph text rather than table text.
                splitTabs = re.split(singSpaceLineRgx, tab, flags = re.M)
                
                for table in splitTabs:
                    meetsReqs = filterRes(table, includeSets = [expList],
                                          excludeSets = [termsToExclude])
                    
                    if meetsReqs:
                        expTabsRaw[pdf].append(table)

### Narrowing Result Set

Now that we've extracted export tables, let's evaluate the performance of our script. For what number of pdfs have we succesfully extracted tables? How many pdfs have no or multiple tables? Have we truncated any of these tables or collected unnecessary data surrounding them?

The first day of this project was essentially a cycle of gradually revising the export-table-extraction script and subsequently consulting the output of this cell. The goal was to minimize tables-per-pdf (such that each pdf hopefully had only one export table) while maximizing the number of pdfs with tables (such that as many pdfs as possible had export tables).

In [91]:
from operator import gt, eq, lt

#Returns list of tablists for which the length
#fulfills the specified 'comp' operation (e.g. 'gt')
# against the 'length' param.
def searchTabByLen(tabDict, comp, length):
    return [pdf for pdf, tabList
            in tabDict.items() 
            if comp(len(tabList), length)]

#How many tables sets in result have a single, multiple, or no tables?
def evaluateTables(tabDict):
    print('\tThere are %d pdfs for which we found multiple results.'
          % len(searchTabByLen(tabDict, gt, 1)))
    print('\tThere are %d pdfs for which we found a single result.'
          % len(searchTabByLen(tabDict, eq, 1)))
    print('\tThere are %d pdfs for which we found no results.'
          % len(searchTabByLen(tabDict, lt, 1)))
    
evaluateTables(expTabsRaw)

	There are 42 pdfs for which we found multiple results.
	There are 6 pdfs for which we found a single result.
	There are 0 pdfs for which we found no results.


At this point, we need to trim down our result set a bit, such that each pdf has only a single result set. After a few hours experimenting with various criteria, I ascertained a relatively effective list. We will be sure to implement more lenient requirements for result sets with only a single table candidate, of course, to avoid depopulating results sets that already have one table. 

Our criteria function to prioritize results which contains units commonly used in export tables (e.g. 'value', 'ton', etc.) and deprioritize terms pertaining to country names. The latter serves to filter out tables which list exports on a country-by-country basis in favor of those which list only commodities and values.

In [92]:
expTabsCut = { pdf : [] for pdf in expTabsRaw }

yearsListRgx = '((19[0-9b ]{2}).{0,2}[\s]+){2,}'
yearsList = [yearsListRgx]
valueTerms = ['value', '£', 'ton']
countryTerms = ['countries', 'union', 'united', 'victoria']
    
for pdf, arr in expTabsRaw.items(): 
    numTabs = len(arr)
    
    for tab in arr:
        #Requirements for a duplicate table to be included in reduced array.
        meetsReqsForDup = filterRes(tab, includeSets = [yearsList, valueTerms],
                                excludeSets = [countryTerms, ['\(b\) Re-exports']])
        
        if numTabs == 1 or meetsReqsForDup:
            expTabsCut[pdf].append(tab)
        
        else:
            numTabs -= 1
            
evaluateTables(expTabsCut)

	There are 3 pdfs for which we found multiple results.
	There are 45 pdfs for which we found a single result.
	There are 0 pdfs for which we found no results.


After investigating the 3 remaining pdfs, I found a heuristic for which we can easily remove table sets with multiple entries; if an array has multiple entries, it is because one lists items in tons while the other lists them in monetary values. As a result, we can achieve optimal results by simply prioritizing results that don't use tonnage.

In [107]:
#Takes an array, removes entry which lists exports in tonnage if array has multiple entries.
def removeTonnage(arr, pdf):
    revisedArr = []
    listLen = len(arr)
    
    if listLen == 1:
        return arr

    for tab in arr:
        #Make sure that we don't eradicate single-el lists.
        if not 'ton' in tab.lower() or listLen == 1:
            revisedArr.append(tab)
        else:
            listLen -= 1
        
    return revisedArr

expTabs = {pdf : removeTonnage(arr, pdf)[0] for
           pdf, arr in expTabsCut.items()}

#searchTabByLen assumes that each pdf will have a list of tables.
#So, for evaluateTables to work, we have to restructure it.
#We're still gonna use expTabs for data operations, obviously.
reworkedExpTabs = { pdf : [tab] for pdf, tab in expTabs.items() }
evaluateTables(reworkedExpTabs)

	There are 0 pdfs for which we found multiple results.
	There are 48 pdfs for which we found a single result.
	There are 0 pdfs for which we found no results.


Now that we've adequately narrowed our results set, let's take a look at our results.

In [111]:
DIVIDER = '\n----------------------------------------------------------------------------------------------------------\n'

for pdf, tab in expTabs.items():
    print(pdf + ':')
    print(tab + DIVIDER)

b31409350.pdf:

                                              1937•                       1938.
                                    Amount.           Value.      Amount.       Value.
                                                        £                         £
Chillies and capsicums        lb.       3,°77               38       9,091          114
Coffee ...       ...  ...    cwt.         908            2,051         251          587
Maize and maize flour         lb.   iji7I»o°5            1,307     161,041          180
Rice ...                                1,235                6     115,801          517
Tobacco:—
  leaf dark-fired     ...     ,,    7,013,224         204,552     6,130,539    178,802
    ,, flue-cured                   1,230,824          35,899     1,241,972     36,224
    ,, air-cured      ...     ,,      746,274          21,766     1,858,129     54,195
  strips dark fired ...       ,,    4,384,886         127,893     3,05U770      89,010
      ,,   flue-cured .

### Parsing Tables

There are two basic types of tables in our results set, each of which must be parsed using a different method. Note that we won't get 100% accuracy with our parsing methods; even with 70 or 80%, we've still substantially expedited our data extraction process.

Below are some examples of each table type.

#### TABLE TYPE 0:
```
                            Slaughter Stock.
            1932.      1933-          1934*           1935-              1936.
   lb.   5,338,800   9,600,000      6,079,400       5,747,400          8,620,000
   £        23,728      62,400         40,536          38,316             57,468
                                  Tobacco.
   lb.    341.055     377,906         374,952            261,794        282,735
   £       10,850      10,010          10,433              6,542          9,44i
                             Cotton {Seed).
   lb.    750,000     207,000         295,800            200,068        124,750
   £        3.125       1,400           2,048              i,459            950
                            Hides and Skins.
   lb.     60,750     439,820         451,850            496,471        600,000
              633       5,910           6,200              8,449         15,000
   £
                                 Wattle Bark.
   lb.      56,000   1,322,000        300,000            939,200        598,080
   £            28       2,653            536              1,825          U335
12                    COLONIAL REPORTS—ANNUAL
                                      Wool.
     lb.   30,000        17,138           18,638         20,000       22,000
     £        490           533              536            750          825
                                      Butter.
     lb.     1 >705        1,210           1,500             3,202     —
     £           85           61              68               199     —
                                   Butter-Fat.
     lb.    22,197       86,690           27,348         21,139       68,000
     £         740        3>973            1,004            856        3,825
                                     Bullion.
     £       1,542         3,9i4           2,608             2,130     3,866
                                   Metallic Tin.                        -
     lb.   187,980     226,912           362,380        406,963      409,248
     £      J1>497      19,665            37,356         39,628       39,35i
```

#### TABLE TYPE 1:

```
                                           1931.                         1932.
            Article.              Value.           Per cent.   Value.            Per cent,
                                    £                             £
Sisal ...     .            ...   707,177            43-0       698,202            31-9
Cotton        .            ...   119,752             7-3       183,747             8-4
Groundnuts                        28,706             1-7       182,010             8-3
Coffee ...                 ...   247,037            15-0       463,597            21-2
Hides and skins                   83,915             51         99,474             4-5
Copra ...                         62,209             3-8        64,694             2-9
Grain, other than rice            34,547             21         34,600             1*6
Sesame                            36,715             2-2        50,130             2-3
Beeswax                           47,010             2-9        31,965             1-4
Ghee.                ,..          11,549             0-7        16,848             0-8
Rice ...                          51,209             31         62,939             2-9
```

```
                                Principal Domestic Exports.
           Article.                 1913.         1929.       1930.      1931.       1932.
Sisal                   tons        20,834       45,728      49,962     55,939      60,554
Groundnuts               99          8,961        7,765      17,333      3,070      15,873
Coffee                    99         1,059        8,857      11,547      9,251      11,362
Cotton ...            centals       49,101      110,821      82,224     54,349      71,888
Copra                   tons         5,477        7,920        7,395     7,234       7,265
Hides and skins           99         3,456        2,549        2,094     2,111       2,718
Grain                   cwt.        44,640      160,924     143,912    260,099     268,135
Sesame ...              tons         1,476        4,256       3,115      3,825       4,811
Beeswax                  99            559          336          189       607         391
Ghee                    cwt.         6,760        9,073       5,860      5,862       7,172
Soap                     99
                                      —             130          981       316         469
Salt                    tons          —           2,999       3,046      2,963       2,428
```



Fortunately, all tables of Type 0 contain the term `slaughter`, so it's relatively simple to disaggregate the two.

In [115]:
from itertools import groupby

#Disaggregates tables into 2 table formats in pdf set.
def tableKeyFunc(tab):
    tabLow = tab[1].lower()
    
    if 'slaughter' in tabLow: 
        return 0
    return 1

groupTabs = groupby(
                sorted(expTabs.items(), key = tableKeyFunc), 
                tableKeyFunc
            )

type0Tabs, type1Tabs = [dict(group) for _, group in groupTabs]

We're almost ready to loop through each of our two table lists (`type0Tabs` and `type1Tabs`), but, first, let's define a data structure to simplify inserts into a multi-dimensional dict and another function that will be helpful in the table-parsing process.

In [116]:
from collections import defaultdict

#Creates a dictionary of depth n to hold instances of given type.
def nestedDict(n, type):
    if n == 1: 
        return defaultdict(type)
    return defaultdict(lambda: nestedDict(n - 1, type))

#Removes prefix from start of string.
def removePrefix(string, prefix):
    if string.startswith(prefix):
        return string[len(prefix):]
    return string[:]

Now, we can parse the type 0 tables. This relatively simple table structure (particularly compared to type 1) with a low degree of diversity between tables in the set, so the cell below doesn't warrant an in-depth explanation. Essentially, we identify the list of years, identify the exports, and go through numerical values row-by-row basis, entering them into a dictionary based on the last export header and the relevant years column.

In [119]:
#Final data structure.
masterExpDict = {}

for pdf, tab in type0Tabs.items():
    header = None
    lineNo = 0
    
    lines = re.split('\n+', tab)
    #Get rid of whitespace at start of line.
    cleanLines = [line.strip() for line in lines]
    yearsLine = [line for line in cleanLines if 
                 re.search(yearRgx, line)]
    
    years = yearsLine[0].split()
    
    #Filter out any titles grabbed when crossing pages.
    #Also filter out any lines listing years.
    filteredLines = filter(lambda line: not (
                            'COLONIAL' in line
                            or 'WAGES' in line
                            or re.search(yearRgx, line)
                        ),
                      cleanLines)
    #Facilitates format data[export][year][unit].
    data = nestedDict(3, str)
    
    for line in filteredLines:
        hasLbs = line.startswith('lb.')
        hasDigit = re.search('\d', line)
        
        #If no digits and no 'lb.', it's a header.
        if not hasLbs and not hasDigit:
            #Grab up until first period.
            export = line.split('.', 1)[0]
            lineNo = 0
            continue
        
        lineNo += 1
        
        #1st line after header (the name of export) has lb vals, 2nd has £s.
        pound = 'lb.' if lineNo == 1 else '£'
        withoutPound = removePrefix(line, pound)
        splitBySpace = withoutPound.split()

        for year, val in zip(years, splitBySpace):
            data[export][year][pound] = val
    
    masterExpDict[pdf] = data

The structure of type 1 tables is far more complex by comparison. In some instances, table headers contain no units (as in Ex. 1) in the header, and, in some instance, they contain multiple units (as in Ex. 2). As a result, our parsing script must prioritize identification of the list of years and list of units within the table header, and the relevant regexes are shown below.

#### EX. 1:

```
      Commodity.          Unit.     1913.         1930.      1931.       1932.       1933.
Sisal                ..    ton      20,834        49,962     55,939      60,554      69,600
Coffee               ••     »        1,059        11,547      9,251      11,362      12,718
Cotton ...           ..cental       49,101        82,224     54,349      71,888     113,677
Gold                  oz. troy      10,032        12,971     15,200      31,030      38,704
Groundnuts           ..   ton        8,961        17,333      3,070      15,873      19,177
Hides and skins             99       3,456         2,094      2,111       2,718       4,140
                                            50
   Commodity.      Unit.      1913.          1930.            1931.        1932.      1933.
Grain               cwt.      44,640        143,912      260,099        268,135       222,658
Copra               ton        5,477          7,395        7,234          7,265         8,157
Beeswax              99          559            189          607            391           680
Ghee                cwt.       6,760          5,860        5,862          7,172         9,604
Sesame ...          ton        1,476          3,115        3,825          4,811         4,441
Ivory               cwt.         212            256          354            517           467
```

#### EX. 2:

```
                                    1931.                 1932.                    1933.
      Commodity.           Value.     Per cent.      Value.    Per cent.     Value.    Per cent.
                              £                      £                         £
Sisal                      707,177      43-0      698,202        31-9       881,772        34-7
Coffee.                    247,037      15-0      463,597        21-2       429,523        16-9
Cotton ...                 119,752       7-3      183,747         8-4       276,864        10-9
Gold   .                    60,183       3-7      157,726         7-2       195,369         7-7
Groundnuts                  28,706       1-7      182,010         8-3       166,223         6-5
Hides and skins             83,915       5-1       99,474         4-5       165,382         6-5
Rice                        51,209       3-1       62,939         2-9        62,382         2-5
Copra ...                   62,209       3-8       64,694         2-9        62,160         2-4
Beeswax                     47,010       2-9       31,965         1-4        52,751         21
Sesame                      36,715       2-2       50,130         2-3        41,845         1-6
Ghee   .                    11,549       0-7       16,848         0-8        19,586         0-8
Grain, other than rice      34,547       2-1       34,600         1-6        17,763         0-7
Ivory ...            ...    13,504       0-8       20,577         0-9        13,753         0-5
```

In [120]:
#PDF data reader does worse with type 2, so we use more general rgx.
yearsRgx = '((19[0-9b ]{2}).+[\s]+.+)+'

#Rgx for numbers with commas separated by a single space.
#Also accounts for various parser misreadings of digits.
spaceSepNums = '([\d,*’SI-]+ ?)+'

#Rgx for units of cells at head of table.
unitRgx = '(.+(Amount|Quantity|Tons|Value|£).+\n?)'

Explaining our entire method for parsing type1 tables would be extremely prolix, so we'll avoid delving into cases where we handle exceptions which encompass only 1 or 2 pdfs. Essentially, I parsed type 1 tables as follows:

- 1) Identify the list of years and the list of units.
- 2) Identify the ratio between the list of years and the list of units.
- 3) Determine from this ratio the relevant structure of the table; for ex. 2 above, for instance, we would generate the structure with the `grouper` function and receive a value of `[(1931, 'Value'), (1931, 'Per cent'), (1932, 'Value'), (1932, 'Per cent')]`.
- 4) Parse the cells of each row, delimited by spaces, right-to-left; right-to-left parsing ensures that we don't interpret noise between the export name and numerical values as data points.
- 5) Store in a master data structure

In [123]:
from itertools import zip_longest

#Split array into arrays of length N.
def grouper(iterable, n, fillvalue = None):
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx
    args = [iter(iterable)] * n
    return zip_longest(fillvalue = fillvalue, *args)

for pdf, tab in type1Tabs.items():
    #Get rid of ellipses or permutations of them caused by pdf reader.
    noEllipse = re.sub('\.\.\.|•••|,,', '', tab)
    data = nestedDict(3, str)
    splitLines = [line.rstrip() for line in
                    re.split('\n+', noEllipse)]
    
    try:
        yearLines = [line for line in splitLines
                    if re.search(yearsRgx, line)]
        yearLine = yearLines[0]
        
        #Deal with specific case that breaks for some reason.
        if '*934-       1935' in yearLine:
            yearLine += yearLines[1]
            
        yearsClean = re.sub(r'(19) (\d{2})', r'\1\2', yearLine)
        
        #Split by spaces, filter out non-numerical values.
        years = [word for word in yearsClean.split()
                 if re.search('\d', word)]

    except:
        print('\t' + pdf + ' failed')
        continue
        
    try:
        unitLines = [res[0] for res in 
                     re.findall(unitRgx, tab)
                     if not re.search('\d', res[0])]
        unitsRaw = max(unitLines, key = len)
        unitsClean = unitsRaw.replace(
                            'Per cent', 'Percent'
                        ).replace('Per cei', 'Percent')
        unitsFiltered = re.sub('Commodity|Article|,|\.', '', unitsClean)
        units = unitsFiltered.split()
        
    except:
        units = []
    
    excRgxs = (['class'] if 'class' in res.lower()
               else [yearsRgx, unitRgx]) 
    
    filteredLines = filter(lambda res: filterRes(
                                        res,
                                        includeSets = [],
                                        excludeSets = [excRgxs]
                                       ),
                           splitLines)
    
    rowsRaw = [item for item in 
                 [re.split(' {2,}', line.lstrip())
                    for line in filteredLines]
               if item != ['']]
    rows = []
    
    alreadyProcessed = 0
    
    for rowInd, row in enumerate(rowsRaw):
        allAlphabetic = all([re.fullmatch('[a-zA-Z, ]+', item)
                             for item in row])
        
        #Account for case where row is just a label, but there are multiple spaces between words
        if allAlphabetic:
            row = [' '.join(row)]
        
        if alreadyProcessed:
            alreadyProcessed = 0
            continue
        
        #Those that start in lowercase are subcategories of another export.
        if not row[0].islower():
            lastCapital = row[0]
        #We gotta account for the one exception where the first char of each row got cut off.
        elif pdf != 'b31411101.pdf':
            row[0] = lastCapital + ' ' + row[0]
        
        #If row only consists of numerical data, it's the result of the previous row getting bifurcated.
        if re.fullmatch('[\d, —]+', ''.join(row)):
            numbers = [cell for cell in row 
                       if re.search('[\d,]+', cell)]
            row = rowsRaw[rowInd - 1] + numbers
            
        #Non-empty elements of row.
        filteredList = list(filter(bool, row))
        cleanList = []
        
        for item in filteredList:
            #We can get cases where numbers separated by a single space are read as one cell.
            #Here, we split them apart to prevent that.
            if re.fullmatch(spaceSepNums, item):
                cleanList.extend(item.split(' '))
                
            #If there's just one item in the row, don't bother with it.
            elif len(filteredList) != 1:
                cleanList.append(item)
            
        rows.append(cleanList)

    for ind, row in enumerate(rows):
        #If row has only one cell, merge with next row
        try:
            if len(row) == 1 and row[0] != '':
                firstNonEmptyInd = [i for i, v in 
                                    enumerate(rows[ind + 1])
                                    if v != ''][0]

                currentVal = rows[ind + 1][firstNonEmptyInd]
                rows[ind + 1][firstNonEmptyInd] = row[0] + currentVal
        
        except:
            continue
    
    #Generate hierarchical structure of table headers.
    #I explain in the previous markdown cell.
    unitYearRatio = int(len(units) / len(years))
    groupedUnits = list(grouper(units, unitYearRatio))
    revGroupedUnits = list(map(reversed, groupedUnits))
    
    if not revGroupedUnits:
        revGroupedUnits = ['£'] * len(years)
    
    #There's usually noise between the export label and values, so we go right-to-left.
    yearUnitZip = zip(reversed(years), revGroupedUnits)
    yearUnitPairs = []
    
    for year, unitList in yearUnitZip:
        yearUnitTups = [(year, unit) for unit in unitList]
        
        for tup in yearUnitTups:
            yearUnitPairs.append(tup)
    
    for row in rows:        
        try:
            export = [cell for cell in row if
                      re.search('[a-zA-Z]{2,}', cell)][0]
        except:
            continue
        
        for cellInd, cell in enumerate(reversed(row)):
            try:
                year, unit = yearUnitPairs[cellInd]
                data[export][year][unit] = cell
            
            except IndexError:
                continue

    masterExpDict[pdf] = data

	b31409386.pdf failed


### DATA REVIEW:

Let's take a look at the data we have so far.

In [122]:
for pdf, exportDict in masterExpDict.items():
    print(pdf)
    for export, dictionary in exportDict.items():
        print('\t' + export)
        for year, values in dictionary.items():
            print('\t\t' + year)
            for unit, value in values.items():
                print('\t\t\t' + unit + ' : ' + value)

b31408990.pdf
	Slaughter Cattle
		1928.
			lb. : 3,639,600
			£ : 54,594
		1929.
			lb. : 5,500,000
			£ : 85,000
		1930.
			lb. : 4,373,700
			£ : 50,763
		1931.
			lb. : 5,181,600
			£ : 33,322
		1932.
			lb. : 5,338,800
			£ : 23,728
	Tobacco
		1928.
			lb. : 1,283,250
			£ : 37,428
		1929.
			lb. : 1,068,400
			£ : 35,613
		1930.
			lb. : 578,330
			£ : 14,397
		1931.
			lb. : 298,413
			£ : 9,082
		1932.
			lb. : 341,055
			£ : 10,850
	Cotton (Seed)
		1928.
			lb. : 850,814
			£ : 13,785
		1929.
			lb. : 2,618.913
			£ : 37,961
		1930.
			lb. : 3,224,182
			£ : 26,868
		1931.
			lb. : 1,532,132
			£ : 9,578
		1932.
			lb. : 750,000
			£ : 3,125
	Hides
		1928.
			lb. : 368,400
			£ : 13,047
		1929.
			lb. : 434,840
			£ : 7,250
		1930.
			lb. : 208,140
			£ : 3,469
		1931.
			lb. : 143,600
			£ : 1,589
		1932.
			lb. : 60,750
			£ : 633
	Skins
		1928.
			lb. : —
			£ : —
		1929.
			lb. : —
			£ : —
		1930.
			lb. : —
			£ : —
		1931.
			lb. : 5,590
			£ : 144
		1932.
			lb. : —
			

There are a handful of errors and pdfs for which our script did not scale well. However, at this point, we seem to have hit a point of diminishing returns. It would be possible to code specific exceptions for each of the 3 or 4 cases which seem to break our rudimentary script, but it would be simpler and easier to do it by hand. Nonetheless, we have managed to reduce several weeks worth of transcription (further complicated by the hierarchical nature of the data, which disallows easy copying into a spreadsheet format) into a few days of code followed by a few hours of revisions (mainly corrected errors by the pdf reader). Given the failings of the pdf reader and diversity of data formats, I see this as a modest accomplishment. However, we're not quite done yet.


### STORING THE DATA

Thusfar, we have neglected to group the data by categories and have instead opted for using pdfs as keys. However, for practical purposes, this will not suffice. We must now design a cursory script to extract the name of the country and year from each report.

In [152]:
countryRgx = '(t?anganyika|swaziland|cameroon|ny ?as ?aland|northern rhodesia|togoland|kenya).{0,200}?(19\d\d)'

#After we search the title page for a country,
#there are several errors we need to correct.
def cleanCountryRes(searchRes):
    if re.search('ny ?as ?aland',
                 searchRes.group(0),
                 re.IGNORECASE):
        return 'NYASALAND'
    elif re.search('t?anganyika',
                   searchRes.group(0),
                   re.IGNORECASE):
        return 'TANGANYIKA'
    
#Takes pdf, returns tuple (country, year).
def getReportData(pdf, pagesDict):
    firstPage = pagesDict[pdf][0]
    countryYearRaw = re.search(countryRgx, firstPage,
                           re.IGNORECASE|re.DOTALL)
    countryRaw = countryYearRaw.group(1)
    country = re.sub('ny ?as ?aland|t?anganyika', 
                     cleanCountryRes, countryRaw,
                     flags = re.IGNORECASE)
    
    year = countryYearRaw.group(2)
    
    return (country, year)

pdfData = { pdf : getReportData(pdf, texts)
             for pdf in masterExpDict.keys() }

pdfData

{'b31408990.pdf': ('SWAZILAND', '1930'),
 'b31409052.pdf': ('SWAZILAND', '1938'),
 'b31409003.pdf': ('SWAZILAND', '1933'),
 'b31409015.pdf': ('SWAZILAND', '1934'),
 'b31408989.pdf': ('SWAZILAND', '1931'),
 'b31409027.pdf': ('SWAZILAND', '1935'),
 'b31409040.pdf': ('SWAZILAND', '1937'),
 'b31409039.pdf': ('SWAZILAND', '1936'),
 'b31409350.pdf': ('NYASALAND', '1938'),
 'b31411514.pdf': ('TOGOLAND', '1938'),
 'b31409106.pdf': ('CAMEROON', '1937'),
 'b31409404.pdf': ('NORTHERN RHODESIA', '1933'),
 'b31409362.pdf': ('NORTHERN RHODESIA', '1929'),
 'b31410327.pdf': ('KENYA', '1938'),
 'b31411472.pdf': ('TOGOLAND', '1934'),
 'b31409064.pdf': ('CAMEROON', '1933'),
 'b3141283x.pdf': ('NYASALAND', '1927'),
 'b31412865.pdf': ('NYASALAND', '1930'),
 'b31409325.pdf': ('NYASALAND', '1935'),
 'b31409398.pdf': ('NORTHERN RHODESIA', '1934'),
 'b31409374.pdf': ('NORTHERN RHODESIA', '1932'),
 'b31411113.pdf': ('TANGANYIKA', '1938'),
 'b3140943x.pdf': ('NORTHERN RHODESIA', '1936'),
 'b31411071.pdf': ('TANG

In [156]:
finalDict = nestedDict(4, str)

for pdf, data in masterExpDict.items():
    country, year = pdfData[pdf]
    finalDict[country][year] = data

One may wonder why we need to disaggregate based on report rather than country or year; if each report contains data from multiple years, there will be overlap. However, due to both inconsistencies between the reports and errors on the part of the pdf reader, this arrangement will not suffice. Instead, we will structure data by report and allow anyone interested in the dataset to choose the datapoints they believe most accurate for any given country and year.

I do intend to go through our programatically generated results and manually revise them. I'll also add in data for a handful of pdfs which were, for various reasons, unreadable by the pdf reader. (These were either cases where pages were stored as images or export tables were stored sideways.) This will be uploaded to github as a separate file. Nonetheless, I'll still show what I was able to generate programatically.

In [157]:
with open('codeGenerated.json', 'w+') as jsonFile:
    json.dump(finalDict, jsonFile)

All in all, pretty decent use of a few days.